In [1]:
import keras
import pandas as pd
import numpy as np

Using TensorFlow backend.
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/akshara/anacon

In [2]:
from keras.layers import Conv1D,Dense
from keras.models import Sequential


In [3]:
x_train = np.load('x_train.npy')
x_test = np.load('x_test.npy')
test_val = np.load('test_val.npy')
y_test = np.load('y_test.npy')
y_train = np.load('y_train.npy')
y_val = np.load('y_val.npy')

In [4]:
y_train.shape

(37065, 20)

In [5]:

len(x_train[3])

1011

In [6]:
import tensorflow as tf
from keras.layers import Dense, Input, Embedding, Lambda, Dropout, Activation, SpatialDropout1D, Reshape, GlobalAveragePooling1D, merge, Flatten, Bidirectional, CuDNNGRU, add, Conv1D, GlobalMaxPooling1D
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras import optimizers
from keras import initializers
from keras.engine import InputSpec, Layer
from keras import backend as K

class AttentionWeightedAverage(Layer):

    def __init__(self, return_attention=False, **kwargs):
        self.init = initializers.get('uniform')
        self.supports_masking = True
        self.return_attention = return_attention
        super(AttentionWeightedAverage, self).__init__(** kwargs)

    def build(self, input_shape):
        self.input_spec = [InputSpec(ndim=3)]
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[2], 1),
                                 name='{}_W'.format(self.name),
                                 initializer=self.init)
        self.trainable_weights = [self.W]
        super(AttentionWeightedAverage, self).build(input_shape)

    def call(self, x, mask=None):
        # computes a probability distribution over the timesteps
        # uses 'max trick' for numerical stability
        # reshape is done to avoid issue with Tensorflow
        # and 1-dimensional weights
        logits = K.dot(x, self.W)
        x_shape = K.shape(x)
        logits = K.reshape(logits, (x_shape[0], x_shape[1]))
        ai = K.exp(logits - K.max(logits, axis=-1, keepdims=True))

        # masked timesteps have zero weight
        if mask is not None:
            mask = K.cast(mask, K.floatx())
            ai = ai * mask
        att_weights = ai / (K.sum(ai, axis=1, keepdims=True) + K.epsilon())
        weighted_input = x * K.expand_dims(att_weights)
        result = K.sum(weighted_input, axis=1)
        if self.return_attention:
            return [result, att_weights]
        return result

    def get_output_shape_for(self, input_shape):
        return self.compute_output_shape(input_shape)

    def compute_output_shape(self, input_shape):
        output_len = input_shape[2]
        if self.return_attention:
            return [(input_shape[0], output_len), (input_shape[0], input_shape[1])]
        return (input_shape[0], output_len)

    def compute_mask(self, input, input_mask=None):
        if isinstance(input_mask, list):
            return [None] * len(input_mask)
        else:
            return None

In [7]:
class squash_function(Layer):
    def squash(s, axis=-1, epsilon=1e-7, name=None):
        with tf.name_scope(name, default_name="squash"):
            squared_norm = tf.reduce_sum(tf.square(s), axis=axis,
                                         keep_dims=True)
            safe_norm = tf.sqrt(squared_norm + epsilon)
            squash_factor = squared_norm / (1. + squared_norm)
            unit_vector = s / safe_norm
            return squash_factor * unit_vector

In [9]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, GlobalMaxPooling1D, Dropout, Conv1D,GlobalAveragePooling1D,Bidirectional,GRU,LSTM,concatenate,Input
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from keras.losses import binary_crossentropy
from keras.optimizers import Adam
from keras_self_attention import SeqSelfAttention

filter_length = 128
num_classes = 20
max_words = 111396
maxlen = 1011

input1 = Input(shape=(maxlen,))
 
x = Embedding(max_words, 20, input_length=maxlen)(input1)

x = squash_function()(x)

x = Bidirectional(GRU(32, return_sequences=True, dropout=0.1,
                                                      recurrent_dropout=0.1))(x)


 
#maxpool = GlobalMaxPooling1D()(x)
x = AttentionWeightedAverage()(x)

x = squash_function()(x)
 
preds = Dense(num_classes, activation="sigmoid")(x)
 
model = keras.Model(input1, preds)
 
model.summary()
 

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])


callbacks = [
    ReduceLROnPlateau(), 
    EarlyStopping(patience=4)
]

history = model.fit(x_train, y_train,
                    epochs=5,
                    batch_size=128,
                    validation_split=0.1,
                    callbacks=callbacks)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1011)              0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 1011, 20)          2227920   
_________________________________________________________________
squash_function_3 (squash_fu (None, 1011, 20)          0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 1011, 64)          10176     
_________________________________________________________________
attention_weighted_average_2 (None, 64)                64        
_________________________________________________________________
squash_function_4 (squash_fu (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                1300

In [10]:
y_pred = model.predict(test_val)
y_pred

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]], dtype=float32)

In [11]:
y_val

array([[0, 0, 0, ..., 0, 0, 1],
       [0, 0, 1, ..., 0, 0, 1],
       [0, 0, 0, ..., 1, 1, 1],
       ...,
       [0, 0, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 1, 1, 0],
       [0, 0, 1, ..., 0, 0, 1]])

In [12]:
y_test.shape

(8177, 20)

In [13]:
print(y_pred[35],y_val[35])

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.] [0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 0]


In [ ]:
(7.8654468e-01+           
5.0963247e-01+
9.6992075e-01+
5.8361495e-01+
3.5057139e-01+
5.7766867e-01)/6

In [14]:
y_pred_round = np.where(y_pred>0.6,1,0)#0.6 enough na lets see....happy ya

###### y_train.shape

In [15]:
from sklearn.metrics import jaccard_score,roc_auc_score,confusion_matrix,hamming_loss
print(jaccard_score(y_val,y_pred_round,average='micro'))
print(jaccard_score(y_val,y_pred_round,average='macro'))
print(roc_auc_score(y_val,y_pred_round))
print(hamming_loss(y_val,y_pred_round))

0.3017913024711341
0.30179130247113417
0.5
0.6982086975288658
